In [17]:
from flask import Flask, request, jsonify
import json
import os
import pyodbc
from datetime import datetime
import shutil
import time
import threading

app = Flask(__name__)

lock = threading.Lock()

# Configuration for JSON output
file_base = "weatherJson_"
archive = "past"
today = datetime.now().strftime("%Y%m%d_%H")
data_folder_path = r"C:\Users\samtr\OneDrive\Documents\Arduino\python\weather station\data"
file = data_folder_path + "\\" + file_base + today + ".json"
print(os.path.join(archive, data_folder_path))
if not os.path.exists(file):
    with open(file, 'x') as f:
        f.write("")
print(file)



# SQL Server connection config (change these to match your setup)
SQL_CONFIG = {
    'server': 'YOUR_SQL_SERVER_HOST',
    'database': 'YourDatabase',
    'username': 'YourUsername',
    'password': 'YourPassword',
    'driver': '{ODBC Driver 17 for SQL Server}'  # Or whatever driver is installed
}

# Connect to SQL Server
def get_sql_connection():
    return pyodbc.connect(
        f"DRIVER={SQL_CONFIG['driver']};"
        f"SERVER={SQL_CONFIG['server']};"
        f"DATABASE={SQL_CONFIG['database']};"
        f"UID={SQL_CONFIG['username']};"
        f"PWD={SQL_CONFIG['password']}"
    )

def check_file():
    global file
    with lock:
        curr_file_time = datetime.now().strftime("%Y%m%d_%H")
        curr_file_path = data_folder_path + "\\" + file_base + curr_file_time +".json"
        if(curr_file_path != file):
            shutil.move(file, os.path.join(archive, data_folder_path))
            file = curr_file_path
            

            
        

# Append data to JSON file
def append_to_json(data, file):
    if not os.path.exists(file):
        with open(file, 'w') as f:
            json.dump([], f)

    with open(file, 'r+') as f:
        entries = json.load(f)
        entries.append(data)
        f.seek(0)
        json.dump(entries, f, indent=4)

# Insert data to SQL Server
def insert_to_sql(data):
    conn = get_sql_connection()
    cursor = conn.cursor()
    
    # Example: Assuming table is called 'SensorData' with columns 'timestamp', 'temperature', 'humidity'
    cursor.execute(
        "INSERT INTO SensorData (timestamp, temperature, humidity) VALUES (?, ?, ?)",
        datetime.now(), data.get('temperature'), data.get('humidity')
    )
    conn.commit()
    conn.close()

def hourly_rotation_checker():
    while True:
        check_file()
        time.sleep(60)

# POST endpoint
@app.route('/data', methods=['POST'])
def receive_data():
    try:
        data = request.json
        if not data:
            return jsonify({'error': 'No JSON payload received'}), 400
        check_file()
        print(file)
        append_to_json(data, file)
        #insert_to_sql(data)
        return jsonify({'status': 'success'}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

threading.Thread(target=hourly_rotation_checker, daemon=True).start()

if __name__ == '__main__':
    check_file()
    app.run(host='0.0.0.0', port=5000)


C:\Users\samtr\OneDrive\Documents\Arduino\python\weather station\data
C:\Users\samtr\OneDrive\Documents\Arduino\python\weather station\data\weatherJson_20250621_09.json
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.50.79:5000
Press CTRL+C to quit


Exception in thread Thread-592:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\samtr\AppData\Local\Temp/ipykernel_38428/3654091935.py", line 87, in hourly_rotation_checker
  File "C:\Users\samtr\AppData\Local\Temp/ipykernel_38428/3654091935.py", line 53, in check_file
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\shutil.py", line 823, in move
    raise Error("Destination path '%s' already exists" % real_dst)
shutil.Error: Destination path 'C:\Users\samtr\OneDrive\Documents\Arduino\python\weather station\data\weatherJson_20250621_09.json' already exists


In [15]:
import os
import json
from flask import Flask, request, jsonify
from datetime import datetime
from threading import Lock, Thread
import time
import shutil

app = Flask(__name__)

DATA_DIR = "data"
ARCHIVE_DIR = "data_archive"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(ARCHIVE_DIR, exist_ok=True)

lock = Lock()
current_filename = None
current_hour = None

def get_current_filename():
    now = datetime.now()
    return now.strftime("%Y-%m-%d_%H.json"), now.hour

def archive_old_file(filename):
    src = os.path.join(DATA_DIR, filename)
    dst = os.path.join(ARCHIVE_DIR, filename)
    if os.path.exists(src):
        shutil.move(src, dst)

def hourly_file_manager():
    global current_filename, current_hour
    while True:
        with lock:
            filename, hour = get_current_filename()
            if filename != current_filename:
                if current_filename:
                    archive_old_file(current_filename)
                current_filename = filename
                current_hour = hour
                file_path = os.path.join(DATA_DIR, current_filename)
                with open(file_path, "w") as f:
                    json.dump([], f)
        time.sleep(60)

@app.route('/data', methods=['POST'])
def submit_data():
    data = request.json
    if not data:
        return jsonify({"error": "No JSON data received"}), 400

    with lock:
        file_path = os.path.join(DATA_DIR, current_filename)
        if not os.path.exists(file_path):
            with open(file_path, "w") as f:
                json.dump([], f)

        with open(file_path, "r+") as f:
            existing_data = json.load(f)
            existing_data.append(data)
            f.seek(0)
            json.dump(existing_data, f, indent=2)

    return jsonify({"status": "success"}), 200

if __name__ == '__main__':
    # Start the hourly file manager in a separate thread
    Thread(target=hourly_file_manager, daemon=True).start()
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.50.79:5000
Press CTRL+C to quit
192.168.50.240 - - [20/Jun/2025 20:00:45] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:00:55] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:00:55] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:06] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:06] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:17] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:17] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:28] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:28] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:38] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:39] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Jun/2025 20:01:49] "POST /data HTTP/1.1" 200 -
192.168.50.240 - - [20/Ju

In [ ]:
file_base = "weatherJson_"
today = datetime.now().strftime("%Y%m%d")
data_folder_path = r"C:\Users\samtr\OneDrive\Documents\Arduino\python\weather station\data"
JSON_FILE = data_folder_path + "\\" + file_base + today
JSON_FILE

'C:\\Users\\samtr\\OneDrive\\Documents\\Arduino\\python\\weather station\\data\\weatherJson_20250619'

In [28]:
with open(JSON_FILE, 'r+') as f:
    entries = json.load(f)

In [29]:
entries

[{'time': 1750359382, 'sensor': 'temperature', 'value': 28.24178},
 {'time': 1750359382, 'sensor': 'humidity', 'value': 51.06874},
 {'time': 1750359393, 'sensor': 'temperature', 'value': 28.25246},
 {'time': 1750359393, 'sensor': 'humidity', 'value': 49.87854},
 {'time': 1750359403, 'sensor': 'temperature', 'value': 28.28718},
 {'time': 1750359403, 'sensor': 'humidity', 'value': 49.77363},
 {'time': 1750359414, 'sensor': 'temperature', 'value': 28.24979},
 {'time': 1750359414, 'sensor': 'humidity', 'value': 49.4036},
 {'time': 1750359425, 'sensor': 'temperature', 'value': 28.26315},
 {'time': 1750359425, 'sensor': 'humidity', 'value': 49.37308},
 {'time': 1750359436, 'sensor': 'temperature', 'value': 28.30854},
 {'time': 1750359436, 'sensor': 'humidity', 'value': 49.41695},
 {'time': 2, 'sensor': 'temperature', 'value': 25.37385},
 {'time': 2, 'sensor': 'humidity', 'value': 57.97917},
 {'time': 14, 'sensor': 'temperature', 'value': 25.2884},
 {'time': 14, 'sensor': 'humidity', 'value':

In [18]:
import pandas as pd 

df = pd.DataFrame(entries)




In [25]:
from time import strftime, localtime

def unix_to_dateTime(unix):
    return strftime('%Y-%m-%d %H:%M:%S', localtime(unix))


df["readTime"] = df["time"].apply(unix_to_dateTime)

In [26]:
df

,time,sensor,value,readTime
0,1750359382,temperature,28.24178,2025-06-19 14:56:22
1,1750359382,humidity,51.06874,2025-06-19 14:56:22
2,1750359393,temperature,28.25246,2025-06-19 14:56:33
3,1750359393,humidity,49.87854,2025-06-19 14:56:33
4,1750359403,temperature,28.28718,2025-06-19 14:56:43
5,1750359403,humidity,49.77363,2025-06-19 14:56:43
6,1750359414,temperature,28.24979,2025-06-19 14:56:54
7,1750359414,humidity,49.40360,2025-06-19 14:56:54
8,1750359425,temperature,28.26315,2025-06-19 14:57:05
9,1750359425,humidity,49.37308,2025-06-19 14:57:05
